# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,27.63,77,100,2.57,US,1691617492
1,1,kailua-kona,19.6406,-155.9956,30.98,66,40,6.69,US,1691617757
2,2,colonia,50.9333,6.9500,16.36,78,20,2.06,DE,1691617387
3,3,novo aripuana,-5.1206,-60.3797,31.80,42,82,0.98,BR,1691617757
4,4,inarajan village,13.2736,144.7484,26.54,95,100,7.91,GU,1691617724


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow Down To Favorable Temperature Range
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] >21)]

# Narrow Daown To Favorable Wind Speed
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] <4.5)]

# Narrow Down Based On Cloudy Conditions (Cloudiness)
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] <1)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,itajobi,-21.3181,-49.0544,25.14,30,0,3.21,BR,1691617765
78,78,changji,44.0167,87.3167,24.44,69,0,1.00,CN,1691617767
265,265,tarija,-21.5355,-64.7296,23.95,31,0,4.12,BO,1691617755
285,285,yigo village,13.5361,144.8886,25.62,94,0,4.12,GU,1691617796
393,393,hammerfest,70.6634,23.6821,23.25,64,0,2.57,NO,1691617675
422,422,nurota,40.5614,65.6886,25.42,35,0,1.44,UZ,1691617818
461,461,andratx,39.5755,2.4202,26.56,74,0,2.84,ES,1691617824
468,468,bekobod,40.2208,69.2697,26.83,32,0,1.61,UZ,1691617825
481,481,bahar,34.9072,48.4414,21.09,32,0,3.09,IR,1691617827
500,500,paracatu,-17.2222,-46.8747,23.24,37,0,1.16,BR,1691617644


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
61,itajobi,BR,-21.3181,-49.0544,30,
78,changji,CN,44.0167,87.3167,69,
265,tarija,BO,-21.5355,-64.7296,31,
285,yigo village,GU,13.5361,144.8886,94,
393,hammerfest,NO,70.6634,23.6821,64,
422,nurota,UZ,40.5614,65.6886,35,
461,andratx,ES,39.5755,2.4202,74,
468,bekobod,UZ,40.2208,69.2697,32,
481,bahar,IR,34.9072,48.4414,32,
500,paracatu,BR,-17.2222,-46.8747,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "apiKey" : geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
itajobi - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
tarija - nearest hotel: Hostal Bolivar
yigo village - nearest hotel: No hotel found
hammerfest - nearest hotel: Thon Hotel Hammerfest
nurota - nearest hotel: Nurota
andratx - nearest hotel: No hotel found
bekobod - nearest hotel: BEK
bahar - nearest hotel: هتل بوعلی
paracatu - nearest hotel: Hotel Presidente
broome - nearest hotel: No hotel found
xiva - nearest hotel: B&B Orzu
bolvadin - nearest hotel: Polybotum Hotel
darwin - nearest hotel: Mantra Pantanas Darwin


,City,Country,Lat,Lng,Humidity,Hotel Name
61,itajobi,BR,-21.3181,-49.0544,30,No hotel found
78,changji,CN,44.0167,87.3167,69,No hotel found
265,tarija,BO,-21.5355,-64.7296,31,Hostal Bolivar
285,yigo village,GU,13.5361,144.8886,94,No hotel found
393,hammerfest,NO,70.6634,23.6821,64,Thon Hotel Hammerfest
422,nurota,UZ,40.5614,65.6886,35,Nurota
461,andratx,ES,39.5755,2.4202,74,No hotel found
468,bekobod,UZ,40.2208,69.2697,32,BEK
481,bahar,IR,34.9072,48.4414,32,هتل بوعلی
500,paracatu,BR,-17.2222,-46.8747,37,Hotel Presidente


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)